In [54]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import xgboost as xgb
import lightgbm as lgb
from datetime import datetime
from catboost import Pool, CatBoostClassifier
from sklearn.metrics import f1_score

In [2]:
train = pd.read_csv("train.csv")
train.set_index('EmployeeNo', inplace=True)
test = pd.read_csv("test.csv")
test.set_index('EmployeeNo', inplace=True)

In [3]:
train.head()

,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not
EmployeeNo,,,,,,,,,,,,,,,,,,
YAK/S/00001,Commercial Sales and Marketing,"MSc, MBA and PhD",Female,Direct Internal process,2,1986,12.5,2011,1,0,41,ANAMBRA,No,Married,No,No,0,0
YAK/S/00002,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,1991,12.5,2015,0,0,52,ANAMBRA,Yes,Married,No,No,0,0
YAK/S/00003,Commercial Sales and Marketing,First Degree or HND,Male,Direct Internal process,2,1987,7.5,2012,0,0,42,KATSINA,Yes,Married,No,No,0,0
YAK/S/00004,Commercial Sales and Marketing,First Degree or HND,Male,Agency and others,3,1982,2.5,2009,0,0,42,NIGER,Yes,Single,No,No,1,0
YAK/S/00006,Information and Strategy,First Degree or HND,Male,Direct Internal process,3,1990,7.5,2012,0,0,77,AKWA IBOM,Yes,Married,No,No,1,0


In [5]:
train['Promoted_or_Not'].head()

EmployeeNo
YAK/S/00001    0
YAK/S/00002    0
YAK/S/00003    0
YAK/S/00004    0
YAK/S/00006    0
Name: Promoted_or_Not, dtype: int64

In [6]:
train['is_train'] = 0
test['is_train'] = 1
train_y = train['Promoted_or_Not']

In [7]:
train = train.drop('Promoted_or_Not',axis=1)

In [8]:
df = pd.concat([train, test], axis=0)

In [9]:
df.head()

,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,is_train
EmployeeNo,,,,,,,,,,,,,,,,,,
YAK/S/00001,Commercial Sales and Marketing,"MSc, MBA and PhD",Female,Direct Internal process,2,1986,12.5,2011,1,0,41,ANAMBRA,No,Married,No,No,0,0
YAK/S/00002,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,1991,12.5,2015,0,0,52,ANAMBRA,Yes,Married,No,No,0,0
YAK/S/00003,Commercial Sales and Marketing,First Degree or HND,Male,Direct Internal process,2,1987,7.5,2012,0,0,42,KATSINA,Yes,Married,No,No,0,0
YAK/S/00004,Commercial Sales and Marketing,First Degree or HND,Male,Agency and others,3,1982,2.5,2009,0,0,42,NIGER,Yes,Single,No,No,1,0
YAK/S/00006,Information and Strategy,First Degree or HND,Male,Direct Internal process,3,1990,7.5,2012,0,0,77,AKWA IBOM,Yes,Married,No,No,1,0


In [10]:
from sklearn.preprocessing import LabelEncoder

# Create dummy variables for every unique category in Division
le = LabelEncoder()
df["Division"] = le.fit_transform(df["Division"])
df["Gender"] = le.fit_transform(df["Gender"])
df["Channel_of_Recruitment"] = le.fit_transform(df["Channel_of_Recruitment"])
df["Foreign_schooled"] = le.fit_transform(df["Foreign_schooled"])
df["Marital_Status"] = le.fit_transform(df["Marital_Status"])
df["Past_Disciplinary_Action"] = le.fit_transform(df["Past_Disciplinary_Action"])
df["State_Of_Origin"] = le.fit_transform(df["State_Of_Origin"])
df["Previous_IntraDepartmental_Movement"] = le.fit_transform(df["Previous_IntraDepartmental_Movement"])
df['Qualification'] = df['Qualification'].fillna('First Degree or HND')
df['Qualification'] = le.fit_transform(df['Qualification'])
df['No_of_previous_employers'] = df['No_of_previous_employers'].replace('More than 5', 6)
df['No_of_previous_employers'] = pd.to_numeric(df['No_of_previous_employers'])

In [11]:
df.head()

,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,is_train
EmployeeNo,,,,,,,,,,,,,,,,,,
YAK/S/00001,1,1,0,1,2,1986,12.5,2011,1,0,41,3,0,0,0,0,0,0
YAK/S/00002,2,0,1,0,2,1991,12.5,2015,0,0,52,3,1,0,0,0,0,0
YAK/S/00003,1,0,1,1,2,1987,7.5,2012,0,0,42,20,1,0,0,0,0,0
YAK/S/00004,1,0,1,0,3,1982,2.5,2009,0,0,42,26,1,2,0,0,1,0
YAK/S/00006,4,0,1,1,3,1990,7.5,2012,0,0,77,2,1,0,0,0,1,0


In [12]:
train_x = df

In [13]:
test_x = train_x[train_x['is_train'] == 1]
# Test before Train !!
train_x = train_x[train_x['is_train'] == 0]

train_x = train_x.drop(['is_train',],axis=1)
test_x = test_x.drop(['is_train',],axis=1)

In [31]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from math import sqrt
import math

rmse = lambda x, y: math.sqrt(mean_squared_error(x, y))

In [57]:
train_pool = Pool(train_x, train_y) 
test_pool = Pool(test_x)
cbm0 = CatBoostClassifier(rsm=0.2, depth=13, learning_rate=0.2)

In [58]:
#XGBRegressor
boost_params = {'eval_metric': 'rmse'}
xgb0 = xgb.XGBRegressor(
    max_depth=13,
    learning_rate=0.1,
    n_estimators=2000,
    objective='binary:logistic',
    gamma=0,
    min_child_weight=1,
    subsample=1,
    colsample_bytree=1,
    scale_pos_weight=1,
    seed=4,
    **boost_params)

#LGBMRegressor
gbm0 = lgb.LGBMRegressor(
    objective='binary',
    num_leaves=65,
    learning_rate=0.2,
    n_estimators=2000)

print("Fitting CatBoostRegressor model...")
cbm0.fit(train_pool)
print("Finished fitting CatBoostRegressor model")

print("Fitting XGBRegressor model...")
xgb0.fit(train_x, train_y)
print("Finished fitting XGBRegressor model")

print("Fitting LGBMRegressor model...")
gbm0.fit(train_x, train_y, eval_metric='rmse')
print("Finished fitting LGBMRegressor model")

Fitting CatBoostRegressor model...
0:	learn: 0.4841583	total: 384ms	remaining: 6m 23s
1:	learn: 0.3778582	total: 1.12s	remaining: 9m 21s
2:	learn: 0.3310870	total: 1.25s	remaining: 6m 56s
3:	learn: 0.2984121	total: 1.42s	remaining: 5m 54s
4:	learn: 0.2739383	total: 1.59s	remaining: 5m 17s
5:	learn: 0.2629986	total: 1.94s	remaining: 5m 21s
6:	learn: 0.2486497	total: 3.78s	remaining: 8m 55s
7:	learn: 0.2366683	total: 4.22s	remaining: 8m 43s
8:	learn: 0.2352966	total: 4.35s	remaining: 7m 59s
9:	learn: 0.2254357	total: 7.9s	remaining: 13m 2s
10:	learn: 0.2215960	total: 8.54s	remaining: 12m 47s
11:	learn: 0.2114704	total: 11.9s	remaining: 16m 16s
12:	learn: 0.2112450	total: 12s	remaining: 15m 10s
13:	learn: 0.2108408	total: 12.2s	remaining: 14m 17s
14:	learn: 0.2018317	total: 12.8s	remaining: 13m 58s
15:	learn: 0.1994409	total: 13s	remaining: 13m 18s
16:	learn: 0.1990941	total: 13.7s	remaining: 13m 10s
17:	learn: 0.1977713	total: 14s	remaining: 12m 42s
18:	learn: 0.1965485	total: 15.7s	rema

153:	learn: 0.1396649	total: 4m 38s	remaining: 25m 30s
154:	learn: 0.1393442	total: 4m 45s	remaining: 25m 59s
155:	learn: 0.1386294	total: 4m 54s	remaining: 26m 34s
156:	learn: 0.1386034	total: 4m 54s	remaining: 26m 23s
157:	learn: 0.1380596	total: 5m 2s	remaining: 26m 54s
158:	learn: 0.1379950	total: 5m 3s	remaining: 26m 43s
159:	learn: 0.1379816	total: 5m 3s	remaining: 26m 32s
160:	learn: 0.1378191	total: 5m 3s	remaining: 26m 23s
161:	learn: 0.1375288	total: 5m 4s	remaining: 26m 15s
162:	learn: 0.1373604	total: 5m 12s	remaining: 26m 44s
163:	learn: 0.1368089	total: 5m 20s	remaining: 27m 15s
164:	learn: 0.1366240	total: 5m 21s	remaining: 27m 4s
165:	learn: 0.1364308	total: 5m 22s	remaining: 27m 2s
166:	learn: 0.1363188	total: 5m 23s	remaining: 26m 54s
167:	learn: 0.1361233	total: 5m 24s	remaining: 26m 44s
168:	learn: 0.1359777	total: 5m 31s	remaining: 27m 10s
169:	learn: 0.1340247	total: 5m 39s	remaining: 27m 38s
170:	learn: 0.1340154	total: 5m 39s	remaining: 27m 27s
171:	learn: 0.134

304:	learn: 0.1218989	total: 8m 57s	remaining: 20m 23s
305:	learn: 0.1218988	total: 8m 57s	remaining: 20m 18s
306:	learn: 0.1218913	total: 8m 57s	remaining: 20m 13s
307:	learn: 0.1217298	total: 9m 5s	remaining: 20m 25s
308:	learn: 0.1216605	total: 9m 5s	remaining: 20m 20s
309:	learn: 0.1216291	total: 9m 5s	remaining: 20m 15s
310:	learn: 0.1214904	total: 9m 13s	remaining: 20m 25s
311:	learn: 0.1214896	total: 9m 13s	remaining: 20m 20s
312:	learn: 0.1214293	total: 9m 13s	remaining: 20m 15s
313:	learn: 0.1214203	total: 9m 13s	remaining: 20m 10s
314:	learn: 0.1214020	total: 9m 15s	remaining: 20m 8s
315:	learn: 0.1207974	total: 9m 23s	remaining: 20m 19s
316:	learn: 0.1206251	total: 9m 23s	remaining: 20m 15s
317:	learn: 0.1205529	total: 9m 24s	remaining: 20m 10s
318:	learn: 0.1205172	total: 9m 24s	remaining: 20m 5s
319:	learn: 0.1204925	total: 9m 24s	remaining: 20m
320:	learn: 0.1204108	total: 9m 26s	remaining: 19m 59s
321:	learn: 0.1201280	total: 9m 34s	remaining: 20m 9s
322:	learn: 0.120102

452:	learn: 0.1106643	total: 13m 22s	remaining: 16m 9s
453:	learn: 0.1106643	total: 13m 23s	remaining: 16m 5s
454:	learn: 0.1106604	total: 13m 23s	remaining: 16m 2s
455:	learn: 0.1106375	total: 13m 26s	remaining: 16m 2s
456:	learn: 0.1104598	total: 13m 33s	remaining: 16m 6s
457:	learn: 0.1103977	total: 13m 35s	remaining: 16m 5s
458:	learn: 0.1103031	total: 13m 39s	remaining: 16m 6s
459:	learn: 0.1099980	total: 13m 46s	remaining: 16m 10s
460:	learn: 0.1098105	total: 13m 54s	remaining: 16m 15s
461:	learn: 0.1097925	total: 13m 55s	remaining: 16m 12s
462:	learn: 0.1097846	total: 13m 56s	remaining: 16m 9s
463:	learn: 0.1097823	total: 13m 56s	remaining: 16m 5s
464:	learn: 0.1097697	total: 13m 57s	remaining: 16m 3s
465:	learn: 0.1097568	total: 13m 58s	remaining: 16m 1s
466:	learn: 0.1097224	total: 14m 5s	remaining: 16m 5s
467:	learn: 0.1096804	total: 14m 5s	remaining: 16m 1s
468:	learn: 0.1096646	total: 14m 6s	remaining: 15m 57s
469:	learn: 0.1096646	total: 14m 6s	remaining: 15m 54s
470:	lear

600:	learn: 0.1018326	total: 18m 53s	remaining: 12m 32s
601:	learn: 0.1018210	total: 18m 54s	remaining: 12m 30s
602:	learn: 0.1018203	total: 18m 54s	remaining: 12m 27s
603:	learn: 0.1017701	total: 18m 55s	remaining: 12m 24s
604:	learn: 0.1017548	total: 18m 55s	remaining: 12m 21s
605:	learn: 0.1017533	total: 18m 55s	remaining: 12m 18s
606:	learn: 0.1017532	total: 18m 55s	remaining: 12m 15s
607:	learn: 0.1017087	total: 18m 55s	remaining: 12m 12s
608:	learn: 0.1017082	total: 18m 56s	remaining: 12m 9s
609:	learn: 0.1017075	total: 18m 56s	remaining: 12m 6s
610:	learn: 0.1017056	total: 18m 56s	remaining: 12m 3s
611:	learn: 0.1017056	total: 18m 56s	remaining: 12m
612:	learn: 0.1017035	total: 18m 56s	remaining: 11m 57s
613:	learn: 0.1016995	total: 18m 57s	remaining: 11m 54s
614:	learn: 0.1016995	total: 18m 57s	remaining: 11m 51s
615:	learn: 0.1016553	total: 19m 2s	remaining: 11m 52s
616:	learn: 0.1016245	total: 19m 3s	remaining: 11m 49s
617:	learn: 0.1016137	total: 19m 3s	remaining: 11m 46s
61

749:	learn: 0.0949683	total: 24m 5s	remaining: 8m 1s
750:	learn: 0.0948348	total: 24m 6s	remaining: 7m 59s
751:	learn: 0.0948118	total: 24m 6s	remaining: 7m 56s
752:	learn: 0.0948097	total: 24m 6s	remaining: 7m 54s
753:	learn: 0.0947902	total: 24m 15s	remaining: 7m 54s
754:	learn: 0.0947559	total: 24m 16s	remaining: 7m 52s
755:	learn: 0.0947475	total: 24m 16s	remaining: 7m 50s
756:	learn: 0.0947452	total: 24m 16s	remaining: 7m 47s
757:	learn: 0.0946310	total: 24m 17s	remaining: 7m 45s
758:	learn: 0.0945750	total: 24m 18s	remaining: 7m 43s
759:	learn: 0.0945698	total: 24m 18s	remaining: 7m 40s
760:	learn: 0.0945689	total: 24m 18s	remaining: 7m 38s
761:	learn: 0.0944140	total: 24m 19s	remaining: 7m 35s
762:	learn: 0.0943599	total: 24m 20s	remaining: 7m 33s
763:	learn: 0.0943584	total: 24m 20s	remaining: 7m 31s
764:	learn: 0.0943441	total: 24m 20s	remaining: 7m 28s
765:	learn: 0.0943436	total: 24m 20s	remaining: 7m 26s
766:	learn: 0.0942106	total: 24m 26s	remaining: 7m 25s
767:	learn: 0.0

899:	learn: 0.0895602	total: 28m 39s	remaining: 3m 11s
900:	learn: 0.0894182	total: 28m 44s	remaining: 3m 9s
901:	learn: 0.0894156	total: 28m 44s	remaining: 3m 7s
902:	learn: 0.0892734	total: 28m 45s	remaining: 3m 5s
903:	learn: 0.0889540	total: 28m 55s	remaining: 3m 4s
904:	learn: 0.0889359	total: 28m 55s	remaining: 3m 2s
905:	learn: 0.0889224	total: 29m 1s	remaining: 3m
906:	learn: 0.0889224	total: 29m 1s	remaining: 2m 58s
907:	learn: 0.0889213	total: 29m 2s	remaining: 2m 56s
908:	learn: 0.0889157	total: 29m 3s	remaining: 2m 54s
909:	learn: 0.0889157	total: 29m 3s	remaining: 2m 52s
910:	learn: 0.0889102	total: 29m 5s	remaining: 2m 50s
911:	learn: 0.0889061	total: 29m 12s	remaining: 2m 49s
912:	learn: 0.0889058	total: 29m 13s	remaining: 2m 47s
913:	learn: 0.0889043	total: 29m 14s	remaining: 2m 45s
914:	learn: 0.0889041	total: 29m 15s	remaining: 2m 43s
915:	learn: 0.0889037	total: 29m 15s	remaining: 2m 41s
916:	learn: 0.0888790	total: 29m 42s	remaining: 2m 41s
917:	learn: 0.0888758	tot

In [59]:
predict_y1 = gbm0.predict(test_x)
predict_y2 = xgb0.predict(test_x)
predict_y3 = cbm0.predict(test_pool)

In [60]:
predict_y = (predict_y1+predict_y2+predict_y3)

In [ ]:
predict_y1

In [64]:
predict_y = [int(round(value)) for value in predict_y]

In [65]:
submit = pd.DataFrame(
   {
       'EmployeeNo':test.index,
       'Promoted_or_Not':predict_y
   }
)


submit.to_csv('submit.csv',index=False)

In [66]:
final = pd.read_csv("submit.csv")
final.head()

,EmployeeNo,Promoted_or_Not
0,YAK/S/00005,0
1,YAK/S/00011,0
2,YAK/S/00015,0
3,YAK/S/00016,0
4,YAK/S/00017,0


In [32]:
# A parameter grid for XGBoost
params = {'min_child_weight':[4,5], 
          'nthread':[4],
          'gamma':[i/10.0 for i in range(3,6)],  
          'learning_rate': [.03, 0.05, .07, 0.1],
          'subsample':[i/10.0 for i in range(6,11)],
          'min_child_weight': [4],
          'silent': [1],
          'colsample_bytree':[i/10.0 for i in range(6,11)], 
          'max_depth': [2,3,4],
          'n_estimators': [1000]}

my_XGB_model = xgb.XGBRegressor(nthread=-1,objective='binary:logistic',missing=None,seed=8)
grid = RandomizedSearchCV(my_XGB_model, params, n_jobs=1,n_iter=10)
grid.fit(train_x, train_y)

c:\program files\python37\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
c:\program files\python37\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
c:\program files\python37\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
c:\program files\python37\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
c:\program files\python37\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if geta

RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=3, min_child_weight=1,
                                          missing=None, n_estimators=100,
                                          n_jobs=1, nthread=-1,
                                          objective='binary:logistic',
                                          r...
                   iid='warn', n_iter=10, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.6, 0.7, 0.8, 0.9,
                                                        

In [36]:
XGB_predictions1 = grid.predict(test_x)
XGB_predictions = [int(round(value)) for value in XGB_predictions1]
submit = pd.DataFrame(
   {
       'EmployeeNo':test.index,
       'Promoted_or_Not':XGB_predictions
   }
)


submit.to_csv('submitxgb.csv',index=False)